In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os,sys, gc
import matplotlib.pyplot as plt
from scipy.stats import entropy
sys.path.append("/content/drive/My Drive/phd/")
sys.path.append("/content/drive/My Drive/phd/hsbm-occam")
from hsbmpy import get_max_available_L

In [ ]:
algorithm = "topsbm"
directory='/content/drive/My Drive/phd/datasets/cancers/lung'
L = get_max_available_L(directory, algorithm)-1
os.chdir(directory)

In [ ]:
label = 'cancer.type'

df_topics = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%(algorithm,algorithm,L)).set_index('doc').drop('i_doc', axis=1)
df_words = pd.read_csv("%s/%s_level_%d_word-dist.csv"%(algorithm,algorithm,L), index_col=0)
df_words.index=[g[:15] for g in df_words.index]
df = pd.read_csv("mainTable.csv", index_col=0).reindex(index=df_words.index)
df = df.divide(df.sum(0),1).transpose().fillna(0)
df_files=pd.read_csv("files.dat", index_col=0)
df_topics.insert(0,'tissue', df_files.reindex(index=df_topics.index)[label])
df_topic_tissue = df_topics.groupby('tissue').mean()
df_files.replace("Discordant_LUSC","LUAD", inplace=True)
df_files.head()

## NN based predictor

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution1D
from tensorflow.keras.losses import binary_crossentropy,mean_squared_error, categorical_crossentropy
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from tensorflow.python.client.device_lib import list_local_devices
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import os,sys, gc
list_local_devices()

In [ ]:
df_labels=df_files.copy()
df_labels=df_labels.reindex(index=df_topics.index)

uniq = len(df_labels[label].unique())

X_train = df_topics.drop('tissue',1)
X_train = X_train.subtract(X_train.mean(0),1).divide(0.5*(X_train.max(0)-X_train.min(0)),1).values.astype(float) #SGD transform
Y_train = to_categorical(np.unique(df_labels[label], return_inverse=True)[1])
inputs = X_train.shape[1]

Y_train = np.argmax(Y_train, axis=-1)
uniq = 1
activation_func = "sigmoid"
lr = 0.01
momentum = 0.95
l1 = 0.01
l2 = 0.0001
loss=binary_crossentropy

classes=np.unique(df_labels[df_labels.index.isin(df.index)][label], return_inverse=True)[0]

X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, random_state=42, train_size=0.8)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
K.clear_session()

os.system("rm -rf log.csv")
csv_logger = CSVLogger('log.csv', append=True, separator=',')
es = EarlyStopping(monitor='val_loss', min_delta=1e-3, mode='min', patience=25)

model=Sequential()
model.add(Dense(units=20, input_dim=inputs, use_bias=True, bias_initializer="ones", activation="relu", kernel_regularizer=l1_l2(l1=l1, l2=l2)))
model.add(Dense(units=uniq, activation=activation_func))
model.compile(loss=loss, optimizer=SGD(lr=lr, momentum=momentum), metrics=['accuracy', 'AUC'])
K.set_learning_phase(0)

print(model.summary())
plot_model(model, to_file=f"model_{label}.png", dpi=600, show_shapes=True)


In [ ]:
with tf.device("GPU"):
    model.fit(X_train, Y_train, epochs=1000, batch_size=50, verbose=1, validation_split=0.4, callbacks=[csv_logger, es], shuffle=True, use_multiprocessing=True, workers=-1)

In [ ]:
pd.read_csv("log.csv", sep=",")[['loss','val_loss']].plot()

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
model.save(f"model_{label}.h5")
#model = load_model(f"model_{label}.h5")

In [ ]:
gc.collect()

# evaluate on non used on topsbm training


In [ ]:
classes = df_files[label].unique()
print(X_test.shape, Y_test.shape)

In [ ]:
model.evaluate(X_test, Y_test, verbose=2, workers=-1, use_multiprocessing=True)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import seaborn as sns

In [ ]:
y_pred_p = model.predict(X_test).ravel()
y_true = Y_test
y_pred = np.ones(y_pred_p.shape)
y_pred[y_pred_p<0.5]=0
results = confusion_matrix(y_true, y_pred, normalize="true")

In [ ]:
cm = sns.clustermap(results, 
                    vmax=1,  
                    row_cluster=False, 
                    col_cluster=False, 
                    xticklabels=classes, 
                    yticklabels=classes, 
                    annot=False,
                    annot_kws={"fontsize":25})
ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_ylabel("real", fontsize=35, rotation=90)
ax.set_yticklabels(labels=classes, rotation=0)
ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")

ax.set_xticklabels(labels=classes, rotation=45)
ax.set_xlabel("predicted",fontsize=35)
ax.tick_params(labelsize=35)

cax = cm.ax_cbar
cax.tick_params(labelsize=30)
cax.set_title("P()", fontsize=30)
plt.tight_layout()
cm.savefig(f"predict_{label}.pdf")

plt.show()

In [ ]:
fig, ax= plt.subplots(figsize=(18,15))

ax.hist(y_pred_p[Y_test==0], histtype="step", lw=15, bins=10, density=True, label=classes[0])
ax.hist(y_pred_p[Y_test==1], histtype="step", lw=15, bins=10, density=True, label=classes[1])

ax.tick_params(labelsize=35)
ax.legend(fontsize=35, loc="upper left")
ax.set_title("", fontsize=35)

ax.set_xlim(-0.05,1.05)

ax.set_xlabel("Z", fontsize=35)
ax.set_ylabel("pdf", fontsize=35)

plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred_p)

In [ ]:
fig, ax = plt.subplots(figsize=(18,15))

ax.plot(fpr,tpr, c="gray", lw=15)

ax.plot([0, 1], [0, 1], 'k--', lw=15)
ax.set_xlabel('False positive rate', fontsize=35)
ax.set_ylabel('True positive rate', fontsize=35)


for f, t, thr in zip(fpr[::100], tpr[::100], thresholds[::100]):
    ax.annotate(thr, (f, t), fontsize=25)

ax.annotate("Area Under Curve = %.4f"%roc_auc_score(y_true, y_pred_p), xy=(0.6, 0.5), fontsize=25)

ax.tick_params(labelsize=35)

plt.show()
fig.savefig(f"roc_{label}.pdf")

# logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, auc

In [ ]:
cv = KFold(n_splits=3)

param_grid = {"C":np.linspace(1,20,10)}
model = LogisticRegression(C=1., fit_intercept=True)

gs = GridSearchCV(model, param_grid, cv = cv, n_jobs=-1)

In [ ]:
gs.fit(X_train, Y_train)
gs.best_params_

In [ ]:
model = gs.best_estimator_

In [ ]:
y_pred = model.predict(X_test)
y_pred_p = model.predict_proba(X_test).T[1]

In [ ]:
accuracy_score(Y_test, y_pred)

In [ ]:
fpr, tpr, thresholds = roc_curve(np.abs(y_true-1), y_pred_p.T[0])
auc(fpr, tpr)